In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cd drive/MyDrive/AmlProject


/content/drive/MyDrive/AmlProject


In [8]:
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pycocotools.coco import COCO
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import Image,ImageDraw
import cv2
import albumentations as A  # our data augmentation library
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm # progress bar
from torchvision.utils import draw_bounding_boxes
from torch.utils.data import TensorDataset,random_split
from albumentations.pytorch import ToTensorV2
import xml.etree.ElementTree as ET
import datasets
import re
import json
from torch.optim import Adam,lr_scheduler
import tqdm

In [4]:
def object_detection_metrics(preds,modtarg):
    resultIou,resultmiss,confval=[],[],[]
    thres = 0.8
    currtru=False
    for i in range(len(modtarg)):
      currtru=False
      levelIou,levelmiss,levelconfval=[],[],[]
      for j in range(len(preds[i]['boxes'])):
        windxl=0
        windxm=0
        windyl=0
        windym=0

        if preds[i]['boxes'][j,0]>modtarg[i]['boxes'][0,0] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,2]:
          windxl=preds[i]['boxes'][j,0]
        elif preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,0]:
          windxl=modtarg[i]['boxes'][0,0]
        else:
          windxl=0

        if preds[i]['boxes'][j,2]>modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2]:
          windxm=modtarg[i]['boxes'][0,2]
        elif preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,0] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2]:
          windxm=preds[i]['boxes'][j,2]
        else:
          windxm=0

        if preds[i]['boxes'][j,1]>modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1]:
          windyl=preds[i]['boxes'][j,1]
        elif preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1]:
          windyl=modtarg[i]['boxes'][0,1]
        else:
          windyl=0

        if preds[i]['boxes'][j,3]>modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3]:
          windym=modtarg[i]['boxes'][0,3]
        elif preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3]:
          windym=preds[i]['boxes'][j,3]
        else:
          windym=0

        overarea = (windxm-windxl)*(windym-windyl)
        predarea = (preds[i]['boxes'][j,2]-preds[i]['boxes'][j,0])*(preds[i]['boxes'][j,3]-preds[i]['boxes'][j,1])
        iou = overarea / modtarg[i]['area']
        if overarea == 0:
          missarea = 1
        else:
          missarea = overarea / predarea

        if iou > thres and preds[i]['labels'][j] == 1 and currtru == False:
          isconf = 'TruePositive'
          currtru = True
        elif iou > thres:
          isconf = 'FalsePositive'
        else:
          isconf = 'FalseNegative'
        levelIou.append(iou)
        levelmiss.append(missarea)
        levelconfval.append(isconf)

      resultIou.append(levelIou)
      resultmiss.append(levelmiss)
      confval.append(levelconfval)
    return resultIou,resultmiss,confval

In [5]:

traintransform = A.Compose([
    A.Resize(500, 500), # our input size can be 600px
    
    A.RandomBrightnessContrast(p=0.1),
    A.ColorJitter(p=0.1),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc',label_fields=["category"]))

testtransform = A.Compose([
    A.Resize(600, 600), # our input size can be 600px
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))


In [10]:
xmllist=os.listdir('archive/annotations')
xmllist.sort()

In [ ]:
xmllist

In [12]:
inp_img=[]
gt_bndbx=[]
gt_label=[]
gt_imgid=[]
gt_area=[]

In [13]:
for annot in xmllist:
    xmltree = ET.parse(os.path.join('archive/annotations',annot))
    root= xmltree.getroot()
    imgname=root.find('filename').text
    img=cv2.imread(f'archive/images/{imgname}')
    xobject=root.find('object')
    imgheight=int(root.find('size').find('height').text)
    imgwidth=int(root.find('size').find('width').text)
    xmin=int(xobject.find('bndbox')[0].text)
    ymin =int(xobject.find('bndbox')[1].text)
    xmax =int(xobject.find('bndbox')[2].text)
    ymax =int(xobject.find('bndbox')[3].text)
    boxes=[[xmin,ymin,xmax,ymax]]
    transformed=traintransform(image=img,bboxes=boxes,category=[1])
    tbox=transformed['bboxes']
    area=(tbox[0][2]-tbox[0][0])*(tbox[0][3]-tbox[0][1])

    inp_img.append(transformed['image'].detach().cpu().numpy())
    gt_bndbx.append(tbox)
    gt_label.append(transformed['category'])
    gt_imgid.append(imgname)
    gt_area.append(area)


In [14]:
def id_maker(imgname):
    subbed=re.sub('Cars','',imgname)
    subbed=re.sub('.png','',subbed)
    return [int(subbed)]

In [15]:
num_imgid=[id_maker(x) for x in gt_imgid]

In [16]:
gt_bndbx=torch.tensor(gt_bndbx)
gt_label=torch.tensor(gt_label)
num_imgid=torch.tensor(num_imgid)
gt_area=torch.tensor(gt_area)
inp_img=torch.tensor(inp_img)/255

In [17]:
licensedataset = TensorDataset(inp_img,gt_bndbx,gt_label,num_imgid,gt_area)

In [18]:
generator1 = torch.Generator().manual_seed(42)

In [19]:
testd,traind =  random_split(licensedataset,[0.3, 0.7], generator=generator1)


In [20]:
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features # we need to change the head
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)

In [ ]:
device = torch.device("cuda")
model = model.to(device)

In [22]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = Adam(params, lr=0.0004, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
scheduler =lr_scheduler.MultiStepLR(optimizer, milestones=[1,2], gamma=0.8)

In [23]:
train_loader = DataLoader(traind, batch_size=4, shuffle=True)

In [24]:
batch = next(iter(train_loader))

In [ ]:
batch[0][0]

In [23]:
modimg=[image for image in batch[0].to(device)]
modtarg=[]
for i in range(len(batch[0])):

  modtarg.append( {
        'boxes':batch[1][i].to(device),
        'labels':batch[2][i].to(device),
        'image_id':batch[3][i].to(device),
        'area':batch[4][i].to(device)
  } )

In [ ]:
modtarg

In [31]:
model.train()
outs=model(modimg,modtarg)

In [32]:
outs


{'loss_classifier': tensor(1.3495, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0., grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.1024, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0042, dtype=torch.float64, grad_fn=<DivBackward0>)}

In [76]:
model.eval()
preds=model(modimg)

In [94]:
resultIou,resultmiss,confval=object_detection_metrics(preds,modtarg)

In [60]:

        resultIou,resultmiss,confval=[],[],[]
        thres = 0.8
        currtru=False
        i =0
        currtru=False
        levelIou,levelmiss,levelconfval=[],[],[]
        j =0
        windxl=0
        windxm=0
        windyl=0
        windym=0
        
        if preds[i]['boxes'][j,0]>modtarg[i]['boxes'][0,0] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,2]:
          windxl=preds[i]['boxes'][j,0]
        elif preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,0]:
          windxl=modtarg[i]['boxes'][0,0]
        else:
          windxl=0

        if preds[i]['boxes'][j,2]>modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,2] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2]:
          windxm=modtarg[i]['boxes'][0,2]
        elif preds[i]['boxes'][j,2] > modtarg[i]['boxes'][0,0] and preds[i]['boxes'][j,0]<modtarg[i]['boxes'][0,2]:
          windxm=preds[i]['boxes'][j,2]
        else:
          windxm=0

        if preds[i]['boxes'][j,1]>modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1]:
          windyl=preds[i]['boxes'][j,1]
        elif preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1]:
          windyl=modtarg[i]['boxes'][0,1]
        else:
          windyl=0

        if preds[i]['boxes'][j,3]>modtarg[i]['boxes'][0,3] and preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3]:
          windym=modtarg[i]['boxes'][0,3]
        elif preds[i]['boxes'][j,3] > modtarg[i]['boxes'][0,1] and preds[i]['boxes'][j,1]<modtarg[i]['boxes'][0,3]:
          windym=preds[i]['boxes'][j,3]
        else:
          windym=0

        overarea = (windxm-windxl)*(windym-windyl)
        predarea = (preds[i]['boxes'][j,2]-preds[i]['boxes'][j,0])*(preds[i]['boxes'][j,3]-preds[i]['boxes'][j,1])
        iou = overarea / modtarg[i]['area']
        missarea = overarea / predarea
        
        if iou > thres and preds[i]['labels'][j] == 1 and currtru == False:
          isconf = 'TruePositive'
          currtru = True
        elif iou > thres:
          isconf = 'FalsePositive'
        else:
          isconf = 'FalseNegative'
        levelIou.append(iou)
        levelmiss.append(missarea)
        levelconfval.append(isconf)

        resultIou.append(levelIou)
        resultmiss.append(levelmiss)
        confval.append(levelconfval)


In [100]:
resultIou


[[tensor(0.9852, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)],
 [tensor(0.9867, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
  tensor(0.5429, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)]]

In [ ]:
all_losses = []
all_losses_dict = []
for i in range(3):
    for step,train_batch in tqdm(enumerate(train_loader)):
        model.train()
        modimg=[image for image in train_batch[0].to(device)]
        modtarg=[]
        print(len(train_batch[0]))
        for i in range(len(train_batch[0])):
          modtarg.append( {
          'boxes':train_batch[1][i].to(device),
          'labels':train_batch[2][i].to(device),
          'image_id':train_batch[3][i].to(device),
          'area':train_batch[4][i].to(device)
        } )
        loss_dict = model(modimg, modtarg) # the model computes the loss automatically if we pass in targets
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        print(f'step {step} loss {loss_dict}')
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        scheduler.step()

In [28]:
all_losses = []
all_losses_dict = []
for i in range(3):
    for step,train_batch in enumerate(train_loader):
        model.train()
        modimg=[image for image in train_batch[0]]
        modtarg=[]
        
        for i in range(len(train_batch[0])):
          modtarg.append( {
          'boxes':train_batch[1][i],
          'labels':train_batch[2][i],
          'image_id':train_batch[3][i],
          'area':train_batch[4][i]
        } )
        loss_dict = model(modimg, modtarg) # the model computes the loss automatically if we pass in targets
        losses = sum(loss for loss in loss_dict.values())
        loss_dict_append = {k: v.item() for k, v in loss_dict.items()}
        loss_value = losses.item()
        print(f'step {step} loss {loss_dict}')
        all_losses.append(loss_value)
        all_losses_dict.append(loss_dict_append)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        scheduler.step()

step 0 loss {'loss_classifier': tensor(0.0646, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0460, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0514, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0125, dtype=torch.float64, grad_fn=<DivBackward0>)}
step 1 loss {'loss_classifier': tensor(0.1686, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.1285, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0274, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0052, dtype=torch.float64, grad_fn=<DivBackward0>)}
step 2 loss {'loss_classifier': tensor(0.0930, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0800, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.0216, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0038, dtype=torch.float64, grad_fn=<DivBackward0>)}
step 3 loss {'loss_classifier': tensor(0.2010, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.3039, gr

KeyboardInterrupt: ignored

In [26]:
logger.write(json.dumps(all_losses_dict))

36867

In [102]:
logger = open('exp2cont1.txt','w')
logger.write(json.dumps(all_losses_dict))

16003

In [27]:
logger.close()

In [25]:
logger = open('expwithadam.txt','w')

In [ ]:
logger.readlines()

In [29]:
test_loader = DataLoader(testd, batch_size=2, shuffle=True)

In [ ]:
full_test_iou,full_test_miss,full_test_confval=[],[],[]
for step,test_batch in tqdm(enumerate(test_loader)):
        
        model.eval()
        
        modimg=[image for image in test_batch[0].to(device)]
        modtarg=[]
        for i in range(len(test_batch[0])):
          modtarg.append( {
          'boxes':test_batch[1][i].to(device),
          'labels':test_batch[2][i].to(device),
          'image_id':test_batch[3][i].to(device),
          'area':test_batch[4][i].to(device)
        } )
          
        preds = model(modimg)
        resultIou,resultmiss,confval=object_detection_metrics(preds,modtarg)

        full_test_iou.append(resultIou)
        full_test_miss.append(resultmiss)
        full_test_confval.append(confval)

        


In [3]:
full_test_iou,full_test_miss,full_test_confval=[],[],[]
for step,test_batch in tqdm(enumerate(test_loader)):
        
        model.eval()
        
        modimg=[image for image in test_batch[0]]
        modtarg=[]
        for i in range(len(test_batch[0])):
          modtarg.append( {
          'boxes':test_batch[1][i],
          'labels':test_batch[2][i],
          'image_id':test_batch[3][i],
          'area':test_batch[4][i]
        } )
          
        preds = model(modimg)
        resultIou,resultmiss,confval=object_detection_metrics(preds,modtarg)

        full_test_iou.append(resultIou)
        full_test_miss.append(resultmiss)
        full_test_confval.append(confval)

        


NameError: ignored

In [118]:
reporter = open('exp2cont1.txt','w')
reporter.write(json.dumps(all_losses_dict))

<function torch.cuda.memory.empty_cache() -> None>

In [ ]:
!pip install attrdict
